In [1]:
!pip install semantic-kernel==1.31.0

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from semantic_kernel import __version__
__version__

'1.39.0'

In [2]:
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from dotenv import load_dotenv

load_dotenv()

#Initializing the kernel
kernel = Kernel()
selectedService = "azureopenai"
service_id = "default"

kernel.add_service(
    AzureChatCompletion(service_id=service_id, 
                        deployment_name=os.getenv("DEPLOYMENT_NAME"),
                        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                        api_key=os.getenv("AZURE_OPENAI_API_KEY"))
)

#### Using the existing Plugin - Joke

In [3]:
# Adding the plugin to the kernel
plugin = kernel.add_plugin(parent_directory="..\..\semantic-kernel\prompt_template_samples", plugin_name="FunPlugin")

In [4]:
from semantic_kernel.functions import KernelArguments

#Invoking the plugin Joke
joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

I tried time traveling to the dinosaur age, but my GPS kept saying, “In 65 million years, turn left at the giant asteroid.”


#### Inline Functions

In [5]:
## Summarize agent
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt="""{{$input}}
Summarize the above content."""

execution_settings=AzureChatPromptExecutionSettings(
    service_id="default",
    ai_model_id="gpt-4.1",
    max_tokens=1000,
    temperature=0.7
)

prompt_template_config = PromptTemplateConfig(
    template=prompt, 
    name="Summarize Agent",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True)
    ],
    execution_settings=execution_settings,
)

In [6]:
summarize_agent=kernel.add_function(
    function_name="summarizeFunc",
    plugin_name="summarizePlugin",
    prompt_template_config=prompt_template_config
)

In [7]:
input_text = """ 
It took five ill-fated conversations with Jean-Paul Sartre before the Vietnamese philosopher Trần Đức Thảo finally broke with French philosophy. Between November 1949 and January 1950, Thảo and Sartre recorded and transcribed their conversations on the relation between Marxism and the new philosophy of existentialism, with the intention of publishing them. Sartre hoped to prove that Marxism and existentialism – of which he was the primary representative – were consistent with one another: reconcilable projects. However, the two men’s exchange of views collapsed before completion, under a series of recriminations. Thảo remained bitter about this, later referring to an ‘insidious campaign’ among Sartre’s ‘disciples’ to paint him as responsible for the failure of this planned project. Today, the conversations are still lost.

For Thảo, their disagreement lay in the fact that Sartre did not recognise Marxism’s philosophical seriousness. For Sartre, traditional Marxism offered an attractive social and political programme but lacked a real or serious philosophical account of being and human nature. Sartre developed his ideas about existentialism out of an ambition to provide the foundations for a new Left-wing philosophy for the 20th century."""

summary = await kernel.invoke(summarize_agent, input=input_text)
print(summary)

Vietnamese philosopher Trần Đức Thảo and French philosopher Jean-Paul Sartre held five conversations between 1949 and 1950, aiming to reconcile Marxism with existentialism. Sartre wanted to show that the two philosophies could be united, but their discussions ended in disagreement and were never published. Thảo blamed Sartre and his followers for the failure, feeling he was unfairly portrayed as responsible. The core dispute was that Thảo believed Sartre did not take Marxism’s philosophical depth seriously, while Sartre saw Marxism as lacking a solid philosophical foundation and developed existentialism to address this gap. Their conversations remain lost.


In [8]:
### Semantic Analysis

prompt="""
Classify the sentiment of the following text.

Return ONLY ONE WORD from this list:
Positive
Negative
Neutral

Do NOT explain.
Do NOT rephrase the text.
Do NOT add any extra words.

Text: 
{{$input_text}}

"""

execution_settings=AzureChatPromptExecutionSettings(
    service_id="default",
    ai_model_id="gpt-4.1",
    max_tokens=5,
)

get_class=PromptTemplateConfig(
    template=prompt,
    name="get_class",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input_text", description="Text to classify", is_required=True)
    ],
    execution_settings=execution_settings,
)



classification_agent = kernel.add_function(
    function_name="classifyFunc",
    plugin_name="classifyPlugin",
    prompt_template_config=get_class
)

In [9]:
text="The new update is okay, but it could be faster."
analysis = await kernel.invoke(classification_agent, input_text=text)
print(analysis)

Neutral


##### ChatPlugin

In [10]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

execution_settings = AzureChatPromptExecutionSettings(
    service_id=service_id,
    max_tokens=2000, 
    temperature=0.7,
)

chat_prompt_template_config = PromptTemplateConfig(
    name="chat",
    template_format="semantic-kernel",
    template=prompt, 
    input_variables=[
        InputVariable(name="user_input", description="User Input", is_required=True),
        InputVariable(name="history", description="Conversation History", is_required=True, ),
    ],
    execution_settings=execution_settings
)

chat_function = kernel.add_function(
    function_name="chat", 
    plugin_name="chatPlugin",
    prompt_template_config=chat_prompt_template_config,
)

In [11]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are an helpful assistant, in giving the book recommendations.")

arguments = KernelArguments(user_input="Hi I am looking for book suggestions", history=ChatHistory)

In [12]:
response = await kernel.invoke(chat_function, arguments=arguments)
print(response)

Function failed. Error: Argument 'history' has a value that doesn't support automatic encoding. Set allow_dangerously_set_content to 'True' for this argument and implement custom encoding, or provide the value as a string.
Something went wrong in function invocation. During function invocation: 'chatPlugin-chat'. Error description: 'Argument 'history' has a value that doesn't support automatic encoding. Set allow_dangerously_set_content to 'True' for this argument and implement custom encoding, or provide the value as a string.'


KernelInvokeException: Error occurred while invoking function: 'chatPlugin-chat'

In [ ]:
chat_history.add_assistant_message(str(response))

In [ ]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [ ]:
await chat("I like fiction, specifically thrillers.")

User: I like fiction, specifically thrillers.
ChatBot: Great choice! Here are some highly recommended thriller novels:

1. **The Girl with the Dragon Tattoo** by Stieg Larsson  
   — A gripping mystery involving hacking, family secrets, and investigative journalism.

2. **Gone Girl** by Gillian Flynn  
   — A psychological thriller about a marriage gone wrong, packed with twists and unreliable narration.

3. **The Silent Patient** by Alex Michaelides  
   — A psychological thriller about a woman who refuses to speak after being accused of her husband’s murder.

4. **The Woman in the Window** by A.J. Finn  
   — A suspenseful story of an agoraphobic woman who believes she’s witnessed a crime in her neighbor’s house.

5. **Before I Go to Sleep** by S.J. Watson  
   — A psychological thriller centered around a woman with amnesia trying to piece together her past.

Would you like recommendations based on a particular subgenre or author, or are you open to any thrillers?


In [ ]:
print(chat_history)

<chat_history><message role="system"><text>You are an helpful assistant, in giving the book recommendations.</text></message><message role="assistant"><text>Hi! I'd be happy to help with book suggestions. Could you tell me a bit about the types of books you enjoy (genre, themes, favorite authors, etc.)? Or are you looking for something new and different?</text></message><message role="user"><text>I like fiction, specifically thrillers.</text></message><message role="assistant"><text>Great choice! Here are some highly recommended thriller novels:

1. **The Girl with the Dragon Tattoo** by Stieg Larsson  
   — A gripping mystery involving hacking, family secrets, and investigative journalism.

2. **Gone Girl** by Gillian Flynn  
   — A psychological thriller about a marriage gone wrong, packed with twists and unreliable narration.

3. **The Silent Patient** by Alex Michaelides  
   — A psychological thriller about a woman who refuses to speak after being accused of her husband’s murder.


##### Controlling the history by adding memory - Semantic Memory

In [13]:
# To use memory we have to add - Memory Storag, Embedding service. 
# Here we are using VolatileMemory Store for storing memory

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

embedding = AzureTextEmbedding(service_id="embedding")
kernel.add_service(embedding)

In [52]:
# pip install semantic-kernel[inmemory]

In [58]:
pip install semantic-kernel[azure] --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for azure-ai-inference>=1.0.0b6 from https://files.pythonhosted.org/packages/4f/0f/27520da74769db6e58327d96c98e7b9a07ce686dff582c9a5ec60b03f9dd/azure_ai_inference-1.0.0b9-py3-none-any.whl.metadata
  Obtaining dependency information for azure-core-tracing-opentelemetry>=1.0.0b11 from https://files.pythonhosted.org/packages/76/5e/97a471f66935e7f89f521d0e11ae49c7f0871ca38f5c319dccae2155c8d8/azure_core_tracing_opentelemetry-1.0.0b12-py3-none-any.whl.metadata
  Obtaining dependency information for azure-search-documents>=11.6.0b4 from https://files.pythonhosted.org/packages/e5/26/ed4498374f9088818278ac225f2bea688b4ec979d81bf83a5355c8c366af/azure_search_documents-11.7.0b2-py3-none-any.whl.metadata
  Obtaining dependency information for azure-cosmos~=4.7 from https://files.pythonhosted.org/packages/4c/46/c0254ea4635d6093085ddd7ba67240eb0d1da145b9313e8bc92c0ff65ac5/azure_cosmos-4.14


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
from semantic_kernel import __version__
__version__

'1.31.0'

In [15]:
import asyncio
import os
# Import the memory components
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore

# Try the most likely import path for your version
try:
    from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding as AzureEmbedding
except ImportError:
    from semantic_kernel.connectors.ai.open_ai import AzureOpenAITextEmbedding as AzureEmbedding

async def main():
    # 1. Configure the Embedding Service 
    # Notice we use 'AzureEmbedding' here, which matches the import above
    embedding_gen = AzureEmbedding(
        deployment_name="text-embedding-ada-002",
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )

    # 2. Build the Memory Object
    memory = SemanticTextMemory(
        storage=VolatileMemoryStore(),
        embeddings_generator=embedding_gen
    )

    # 3. Save Information
    print("Saving information...")
    await memory.save_information(
        collection="finance_memory",
        id="info1",
        text="Your budget for 2024 is $100,000"
    )

    # 4. Search
    query = "What is my budget?"
    print(f"Searching for: '{query}'")
    results = await memory.search("finance_memory", query, limit=1)

    if results:
        print(f"Found: {results[0].text}")
        print(f"Relevance: {results[0].relevance}")
    else:
        print("No results found. Check your API keys and endpoint.")

# In a Jupyter Notebook / VS Code Interactive cell, use:
await main()

C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\4083502688.py:24: DeprecationWarning: This class will be removed in a future version. Please use the InMemoryStore and Collection instead.
  storage=VolatileMemoryStore(),
C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\4083502688.py:23: DeprecationWarning: This class will be removed in a future version.
  memory = SemanticTextMemory(


Saving information...
Searching for: 'What is my budget?'
Found: Your budget for 2024 is $100,000
Relevance: 0.8299683359427587


In [17]:
import asyncio
import os
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore

# 1. Handle Versioning for Azure AI Search Connector
try:
    # Modern v1.x path
    from semantic_kernel.connectors.ai.azure_ai_search import AzureAISearchMemoryStore
except ImportError:
    try:
        # Legacy/Early v1 path
        from semantic_kernel.connectors.memory.azure_ai_search import AzureAISearchMemoryStore
    except ImportError:
        AzureAISearchMemoryStore = None

# 2. Handle Versioning for OpenAI Embeddings
try:
    from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding as AzureEmbedding
except ImportError:
    from semantic_kernel.connectors.ai.open_ai import AzureOpenAITextEmbedding as AzureEmbedding

async def main():
    # --- STEP 1: Setup Embedding Service ---
    # Replace these strings with your actual values if not using environment variables
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    deployment = "text-embedding-ada-002"

    embedding_gen = AzureEmbedding(
        deployment_name=deployment,
        endpoint=endpoint,
        api_key=api_key,
    )

    # --- STEP 2: Use VolatileStore (RAM) for Finance Data ---
    print("--- PHASE 1: LOCAL MEMORY (VOLATILE) ---")
    memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)

    finance_records = [
        ("budget_2024", "Your budget for 2024 is $100,000"),
        ("savings_2023", "Your savings from 2023 are $50,000"),
        ("investments", "Your investments are $80,000"),
    ]

    for record_id, text in finance_records:
        await memory.save_information(collection="finance", id=record_id, text=text)
        print(f"Saved to RAM: {record_id}")

    # Test Local Search
    query = "What is my budget for 2024?"
    print(f"\ Searching RAM for: '{query}'")
    results = await memory.search("finance", query, limit=1)
    if results:
        print(f"Answer: {results[0].text} (Score: {results[0].relevance:.2f})")

    # --- STEP 3: Handle GitHub Files Logic ---
    print("\n--- PHASE 2: GITHUB FILES ---")
    github_files = [
        {
            "url": "https://github.com/microsoft/semantic-kernel/blob/main/README.md",
            "text": "README: Installation, getting started, and how to contribute",
        },
        {
            "url": "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb",
            "text": "Jupyter notebook describing how to get started with Semantic Kernel",
        }
    ]

    for file in github_files:
        # Saving information using URL as ID to prevent duplicates
        await memory.save_information(
            collection="github", 
            id=file["url"], 
            text=f"URL: {file['url']} Description: {file['text']}"
        )
    
    # Test GitHub Search
    ask = "How should I get started with notebooks?"
    print(f" Searching GitHub Memory: '{ask}'")
    github_results = await memory.search("github", ask, limit=1)
    if github_results:
        print(f"Found: {github_results[0].text}")

    # --- STEP 4: Switch to Azure AI Search (Cloud) ---
    print("\n--- PHASE 3: CLOUD MEMORY (AZURE AI SEARCH) ---")
    search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
    search_key = os.getenv("AZURE_AI_SEARCH_API_KEY")

    if AzureAISearchMemoryStore and search_endpoint and search_key:
        # Note: Azure AI Search collection names must be lowercase with hyphens (no underscores)
        cloud_storage = AzureAISearchMemoryStore(endpoint=search_endpoint, api_key=search_key)
        cloud_memory = SemanticTextMemory(storage=cloud_storage, embeddings_generator=embedding_gen)

        print("Syncing GitHub files to Cloud...")
        await cloud_memory.save_information(
            collection="github-docs", 
            id="readme-main", 
            text="Cloud Copy: Installation and contribution guide"
        )

        cloud_search = await cloud_memory.search("github-docs", "installation", limit=1)
        if cloud_search:
            print(f"Cloud Result: {cloud_search[0].text}")
    else:
        print("Skipping Cloud Phase: Azure AI Search credentials not found or package not compatible.")

# Execute in Notebook
await main()

--- PHASE 1: LOCAL MEMORY (VOLATILE) ---


C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\2532257756.py:38: DeprecationWarning: This class will be removed in a future version. Please use the InMemoryStore and Collection instead.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)
C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\2532257756.py:38: DeprecationWarning: This class will be removed in a future version.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)


Saved to RAM: budget_2024
Saved to RAM: savings_2023
Saved to RAM: investments
\ Searching RAM for: 'What is my budget for 2024?'
Answer: Your budget for 2024 is $100,000 (Score: 0.90)

--- PHASE 2: GITHUB FILES ---
 Searching GitHub Memory: 'How should I get started with notebooks?'
Found: URL: https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb Description: Jupyter notebook describing how to get started with Semantic Kernel

--- PHASE 3: CLOUD MEMORY (AZURE AI SEARCH) ---
Skipping Cloud Phase: Azure AI Search credentials not found or package not compatible.


In [18]:
import asyncio
import os
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore

# Fallback imports for version compatibility
try:
    from semantic_kernel.connectors.ai.azure_ai_search import AzureAISearchMemoryStore
except ImportError:
    try:
        from semantic_kernel.connectors.memory.azure_ai_search import AzureAISearchMemoryStore
    except ImportError:
        AzureAISearchMemoryStore = None

try:
    from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding as AzureEmbedding
except ImportError:
    from semantic_kernel.connectors.ai.open_ai import AzureOpenAITextEmbedding as AzureEmbedding

# --- Helper Function: search_memory_examples ---
async def search_memory_examples(memory_store: SemanticTextMemory, collection: str, questions: list[str]) -> None:
    for question in questions:
        print(f"Question: {question}")
        # search returns a list of MemoryQueryResult
        results = await memory_store.search(collection, question, limit=1)
        
        if results:
            for result in results:
                print(f"Answer: {result.text}")
                print(f"Score: {result.relevance}\n")
        else:
            print("Answer: No relevant information found.\n")

async def main():
    # 1. Setup
    embedding_gen = AzureEmbedding(
        deployment_name="text-embedding-ada-002",
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )

    # Use VolatileStore for this demonstration
    memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)

    # 2. Seed Data (Replicating your GitHub example)
    print("Seeding Memory Store...")
    github_data = [
        ("https://github.com/sk/readme", "README: Installation, getting started, and how to contribute"),
        ("https://github.com/sk/notebook00", "Jupyter notebook describing how to get started with Semantic Kernel"),
        ("https://github.com/sk/notebook02", "Jupyter notebook describing how to pass prompts from a file")
    ]

    for url, content in github_data:
        await memory.save_information(collection="github-files", id=url, text=content)

    # 3. Search (The part you requested)
    print("--- Starting Memory Search ---\n")
    await search_memory_examples(
        memory,
        collection="github-files",
        questions=[
            "What is Semantic Kernel?",
            "How do I get started on it with notebooks?",
            "Where can I find more info on prompts?",
        ]
    )

# Run in Notebook
await main()

C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\2636711275.py:43: DeprecationWarning: This class will be removed in a future version. Please use the InMemoryStore and Collection instead.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)
C:\Users\CheekireddyDhaminiMA\AppData\Local\Temp\ipykernel_35880\2636711275.py:43: DeprecationWarning: This class will be removed in a future version.
  memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)


Seeding Memory Store...
--- Starting Memory Search ---

Question: What is Semantic Kernel?
Answer: Jupyter notebook describing how to get started with Semantic Kernel
Score: 0.8904076405981691

Question: How do I get started on it with notebooks?
Answer: Jupyter notebook describing how to get started with Semantic Kernel
Score: 0.8259922528057194

Question: Where can I find more info on prompts?
Answer: Jupyter notebook describing how to pass prompts from a file
Score: 0.8274885463096238

